* Total questions: 52,060
* Total answers: 55,304
* Top 99 users' average reputation: 11893.464646464647

In [ ]:
from lxml import etree
def parse(data, attr):
    try:
        root = etree.fromstring(data)
        target = root.attrib.get(attr)
    except:
        return False
    return(target)

In [ ]:
posts = data3
users = sc.textFile(localpath('spark-stats-data/allUsers/'))

In [ ]:
good_users = users.map(lambda x: (parse(x, 'Reputation'),parse(x, 'AccountId'))) \
     .sortByKey(ascending = False) \
     .take(99)

good_users = sc.parallelize(good_users)

In [ ]:
postsByUsers = good_users.join(posts)

In [ ]:
good_users.count()

In [ ]:
def answer_percentage():
    return [(7071, 0.9107142857142857)] * 100

grader.score(question_name='spark__answer_percentage', func=answer_percentage)

## Post counts


If we use the total number of posts made on the site as a metric for tenure, we can look at the differences between "younger" and "older" users. You can imagine there might be many interesting features - for now just return the top 100 post counts among all users (of all types of posts) and the average reputation for every user who has that count.

In other words, aggregate the cases where multiple users have the same post count.


#### Checkpoints

* Mean of top 100 post counts: 281.51

In [ ]:
def post_counts():
    return [(118, 3736.5)] * 100

grader.score(question_name='spark__post_counts', func=post_counts)

## Quick answers


How long do you have to wait to get your question answered? Look at the set of ACCEPTED answers which are posted less than three hours after question creation. What is the average number of these "quick answers" as a function of the hour of day the question was asked? You should normalize by how many total accepted answers are garnered by questions posted in a given hour, just like we're counting how many quick accepted answers are garnered by questions posted in a given hour, eg. (quick accepted answers when question hour is 15 / total accepted answers when question hour is 15).

Return a list, whose *i*th element correspond to *i*th hour (e.g. 0 -> midnight, 1 -> 1:00, etc.)

*Note*: When using the `SimpleDateFormat` class in Scala, it's important to account for your machine's local time zone. Our policy will be to use GMT: `hourFormat.setTimeZone(TimeZone.getTimeZone("GMT"))`

*Consider*: What biases are present in our result that we don't account for? How should we handle this?


* Total quick accepted answers: 8,468
* Total accepted answers: 17,096

In [ ]:
downvotes.count()

In [ ]:
fav_up = favorites.join(upvotes) \
                  .map(lambda x: (x[1][0],x[1][1])) \
                  .reduceByKey(lambda x,y: x+y)

fav_down = favorites.join(downvotes) \
                  .map(lambda x: (x[1][0],x[1][1])) \
                  .reduceByKey(lambda x,y: x+y)


In [ ]:
fav_down.take(1)

In [ ]:
ratio_by_fav = fav_up.fullOuterJoin(fav_down) \
              .map(lambda x: (x[0], (0 if x[1][0] == None else x[1][0], 0 if x[1][1] == None else x[1][1]))) \
              .map(lambda x: (x[0],x[1][0]*1.0/(x[1][0]+x[1][1]))) \
              .sortByKey()


In [ ]:
ratio_by_fav.count()

In [ ]:
ratio.takeOrdered(50, key = lambda x: x[0])

In [ ]:
def upvote_percentage():
    return ratio_by_fav.takeOrdered(50, key = lambda x: x[0])
    #return [(20, 0.9952153110047847)] * 50

grader.score(question_name='spark__upvote_percentage', func=upvote_percentage)

In [ ]:
from lxml import etree
x = votes3.take(1)
str(x)[-1]
root = etree.fromstring(str(x)[5:-2])

## Answer percentage


Investigate the correlation between a user's reputation and the kind of posts they make. For the 99 users with the highest reputation, single out posts which are either questions or answers and look at the percentage of these posts that are answers: *(answers / (answers + questions))*. 

Return a tuple of their **user ID** and this fraction.

You should also return (-1, fraction) to represent the case where you average over all users (so you will return 100 entries total).

Again, you only need to run this on the statistics overflow set.


In [ ]:
def quick_answers():
    return [0.] * 24

grader.score(question_name='spark__quick_answers', func=quick_answers)

## Quick answers&mdash;full


Same as above, but on the full Stack Exchange data set.

No pre-parsed data is available for this question.


* Total quick accepted answers: 3,700,224
* Total accepted answers: 5,086,888

## Bad XML


A simple question to test your parsing code. Create an RDD of Post objects where each Post is a valid row of XML from the Cross-Validated (stats.stackexchange.com) `allPosts` data set.

We are going to take several shortcuts to speed up and simplify our computations.  First, your parsing function to only attempt to parse rows that start with `  <row` as these denote actual data entries. This should be done in Spark as the data is being read in from disk, without any pre-Spark processing. 

Return the total number XML rows that started with ` <row` that were subsequently **rejected** during your processing.  Note that the text is unicode, and contains non-ASCII characters.  You may need to re-encode to UTF-8 (depending on your XML parser)

Note that this cleaned data set will be used for all subsequent questions.

*Question*: Can you figure out what filters you need to put in place to avoid throwing parsing errors entirely?

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "temp")

In [ ]:
import os, time
def localpath(path):
    return 'file://' + os.path.join(os.path.abspath(os.path.curdir), path)

In [ ]:
data = sc.textFile(localpath('spark-stats-data/allPosts/'))

In [ ]:
def goodStart(line):
    line = line.strip()
    if line[:4] == '<row':
        return True
    return False

def goodEnd(line):
    line = line.strip()
    if line[-2:] == '/>':
        return True
    return False
    
x = '    <row AcceptedAnswerId="1'
goodEnd(x)

In [ ]:
data2 = data.filter(lambda x: goodStart(x))
data3 = data2.filter(lambda x: goodEnd(x))  # use this for all questions
data2.count() - data3.count()
data3.count()

In [ ]:
def bad_xml():
    return 781

grader.score(question_name='spark__bad_xml', func=bad_xml)

## Upvote percentage


Each post on Stack Exchange can be upvoted, downvoted, and favorited. One "sanity check" we can do is to look at the ratio of upvotes to downvotes (referred to as `UpMod` and `DownMod` in the schema) as a function of how many times the post has been favorited.

You might hypothesize, for example, that posts with more favorites should have a higher upvote/downvote ratio.

Instead of looking at individual posts, we'll aggregate across number of favorites by using the post's number of favorites as our key. Since we're computing ratios, bundling together all posts with the same number of favorites effectively averages over them.  Calculate the average percentage of upvotes *(upvotes / (upvotes + downvotes))* for the smallest 50 **keys**.

Do the analysis on the smaller Cross-Validated data set.


* Total upvotes: 313,819
* Total downvotes: 13,019
* Mean of first 50 keys (averaging the keys themselves): 24.76

In [ ]:
def quick_answers_full():
    return [0.] * 24

grader.score(question_name='spark__quick_answers_full', func=quick_answers_full)

## Identify veterans


It can be interesting to think about what factors influence a user to remain active on the site over a long period of time. In order not to bias the results towards older users, we'll define a time window between 100 and 150 days after account creation. If the user has made a post in this time, we'll consider them active and well on their way to being veterans of the site; if not, they are inactive and were likely brief users.

*Consider*: What other parameterizations of "activity" could we use, and how would they differ in terms of splitting our user base?

*Consider*: What other biases are still not dealt with, after using the above approach?

Let's see if there are differences between the first ever question posts of "veterans" vs. "brief users". For each group separately, average the score, views, number of answers, and number of favorites of the users' **first question**.

*Consider*: What story could you tell from these numbers? How do the numbers support it?


* Total brief users: 24,864
* Total veteran users: 2,027

In [ ]:
def identify_veterans():
    return {"vet_score": 0,
            "vet_views": 0,
            "vet_answers": 0,
            "vet_favorites": 0,
            "brief_score": 0,
            "brief_views": 0,
            "brief_answers": 0,
            "brief_favorites": 0
           }

grader.score(question_name='spark__identify_veterans', func=identify_veterans)

## Identify veterans&mdash;full


* Total brief users: 1,848,628
* Total veteran users: 288,285

In [ ]:
def identify_veterans_full():
    return {"vet_score": 0,
            "vet_views": 0,
            "vet_answers": 0,
            "vet_favorites": 0,
            "brief_score": 0,
            "brief_views": 0,
            "brief_answers": 0,
            "brief_favorites": 0
           }

grader.score(question_name='spark__identify_veterans_full', func=identify_veterans_full)

## Word2vec


Word2Vec is an alternative approach for vectorizing text data. The vectorized representations of words in the vocabulary tend to be useful for predicting other words in the document, hence the famous example "vector('king') - vector('man') + vector('woman') ~= vector('queen')".

Let's see how good a Word2Vec model we can train using the tags of each Stack Exchange post as documents (this uses the full data set). Use the implementation of Word2Vec from Spark ML (this will require using DataFrames) to return a list of the top 25 closest synonyms to "ggplot2" and their similarity score in tuple format ("string", number).


#### Parameters


The dimensionality of the vector space should be 100. The random seed should be 42L.


* Mean of the top 25 cosine similarities: 0.7785175901170094

## Accessing the data


The data is available on S3 (s3://dataincubator-course/spark-stack-data). There are three sub-folders, `allUsers`, `allPosts`, and `allVotes`, which contain Gzipped XML with the following format:

```
<row Body="&lt;p&gt;I always validate my web pages, and I recommend you do the same BUT many large company websites DO NOT and cannot validate because the importance of the website looking exactly the same on all systems requires rules to be broken. &lt;/p&gt;&#10;&#10;&lt;p&gt;In general, valid websites help your page look good even on odd configurations (like cell phones) so you should always at least try to make it validate.&lt;/p&gt;&#10;" CommentCount="0" CreationDate="2008-10-12T20:26:29.397" Id="195995" LastActivityDate="2008-10-12T20:26:29.397" OwnerDisplayName="Eric Wendelin" OwnerUserId="25066" ParentId="195973" PostTypeId="2" Score="0" />
```

Data from the much smaller stats.stackexchange.com is available in the same format on S3 (s3://dataincubator-course/spark-stats-data). This site, Cross-Validated, will be used below in some instances to avoid working with the full data set for every question.

The full schema is available as a text file:

In [ ]:
!aws s3 cp s3://dataincubator-course/spark-stats-data/stack_exchange_schema.txt .

You can either get the data by running the appropriate S3 commands in the terminal, or by running this block for the smaller stats data set:

In [ ]:
!mkdir -p spark-stats-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data
!aws s3 sync --exclude '*' --include 'posts*zip' s3://dataincubator-course/spark-stats-data/ ./spark-stats-data

And to get the much larger full data set (be warned, this can take 20 or more minutes, so you may want to run it in the terminal to avoid locking up the notebook):

In [ ]:
!mkdir -p spark-stack-data
!aws s3 sync --exclude '*' --include 'all*' s3://dataincubator-course/spark-stack-data/ ./spark-stack-data

## Data input and parsing


Some rows are split across multiple lines; these can be discarded. Incorrectly formatted XML can also be ignored. It is enough to simply skip problematic rows, the loss of data will not significantly impact our results on this large data sets.

You will need to handle XML parsing yourself using the `\` selector in Scala or something like `lxml.etree` in Python. *Warning*: The built-in `xml.etree.ElementTree` behaves differently and the results don't correspond perfectly with the Scala equivalent.

To make your code more flexible, it's also recommended to incorporate command-line arguments that specify the location of the input data and where output should be written.

The goal should be to have a parsing function that can be applied to the input data to access any XML element desired. It is suggested to use a class structure so that you can create RDDs of Posts, Votes, Users, etc.

``` scala
// Command line arguments in Scala

object Main {
 def main(args: Array[String]) {
   val inputDir = args(0)
   val outputDir = args(1)
   ...
```

``` python

# Command line arguments using sysv or argparse in Python

if __name__ == '__main__':
    main(ARGS.input_dir, ARGS.output_dir)
```

Dates are parsed by default using the Long data type and Unix time (epoch time). In Java/Scala, a given timestamp represents the number of milliseconds since 1970-01-01T00:00:00Z. Also be wary of integer overflow when dealing with Longs. For example, these two are not equal:

`val year: Long = 365 * 24 * 60 * 60 * 1000`

`val year: Long = 365 * 24 * 60 * 60 * 1000L`

## Questions

In [ ]:
def word2vec():
    return [("data.frame", 0.7900882217638416)] * 25

grader.score(question_name='spark__word2vec', func=word2vec)

## Classification


We'd like to see if we can predict the tags of a question from its body text. Technically this is a multi-label classification problem, but to simplify things we'll use a one-vs-all approach where we choose the top k most common tags and train k binary classifiers where the labels indicate the presence or absence of that tag.

Use a logistic regression model as your classifier.

Since we can't reliably save and load models, return a list of 100 tuples ("string", [number, number, number,...]) where "string" is the tag and the numbers are your model's predicted probabilities for class 0 (eg. 0.2 means a prediction that the tag is present) across the test set.

Note that this will require some digging into the result DataFrame to extract.
The length of these probability lists is equal to the length of the test set: 4649.


* The training and tests sets were downloaded with the stats data at the beginning of the notebook.  You can also get them from S3:
  * s3://dataincubator-course/spark-stats-data/posts_train.zip
  * s3://dataincubator-course/spark-stats-data/posts_test.zip
* Tokenize the body text into words
* number of tags to consider k = 100